In [ ]:
# ----------------------------------------------------------------------
# 라이브러리 및 데이터 준비
# ----------------------------------------------------------------------
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest

# 정상 데이터 포인트 (대부분 10 주변에 집중)
rng = np.random.RandomState(42)
X = 0.3 * rng.randn(100, 2)
X_train = np.r_[X + 2, X - 2] # 2개의 군집 생성

# 명확한 이상치 추가 (데이터셋에서 멀리 떨어진 지점)
X_outliers = rng.uniform(low=-4, high=4, size=(20, 2))
X_data = np.r_[X_train, X_outliers] # 학습 데이터 + 이상치

df = pd.DataFrame(X_data, columns=['Feature1', 'Feature2'])
print("--- 원본 데이터셋 (정상 + 이상치) ---")
print(df.head())

In [ ]:
# ----------------------------------------------------------------------
# 2단계: Isolation Forest 모델 학습
# ----------------------------------------------------------------------
# contamination=0.15: 전체 데이터의 약 15%가 이상치라고 가정하고 모델을 학습 (총 240개 중 20개의 이상치, 약 8.3%지만 예시로 15% 사용)
model = IsolationForest(contamination=0.15, random_state=42)

# 모델 학습 (X_data의 패턴을 학습)
model.fit(X_data)

print("\n--- 2단계: 모델 학습 완료 ---")

In [ ]:
# ----------------------------------------------------------------------
# 3단계: 이상치 예측 및 점수 확인
# ----------------------------------------------------------------------

# 1) 예측: 각 데이터 포인트에 대해 이상치 여부 예측
#    결과: 1 (정상), -1 (이상치)
predictions = model.predict(X_data)

# 2) 이상치 점수 (Anomaly Score) 계산
#    낮은 점수일수록 이상치일 확률이 높음(0보다 작을수록 이상치 가능성 큼)
scores = model.decision_function(X_data)

# 결과를 DataFrame에 추가
df['anomaly_prediction'] = predictions
df['anomaly_score'] = scores

print("\n--- 3단계: 예측 및 점수 결과 (상위 5개) ---")
print(df.head()) # 대부분 1(정상)과 높은 점수(덜 이상함)

In [ ]:
# ----------------------------------------------------------------------
# 4단계: 이상치 필터링 및 확인
# ----------------------------------------------------------------------
# 예측값이 -1인 데이터만 필터링하여 이상치 확인
outliers = df[df['anomaly_prediction'] == -1].sort_values(by='anomaly_score')

print("\n--- 4단계: 탐지된 이상치 확인 (점수가 낮은 순 = 가장 이상함) ---")
print(outliers.head())

# 해석: anomaly_score가 0에 가까울수록(또는 음수일수록) 이상치일 가능성이 높고,
#       predictions가 -1이면 모델이 이상치로 분류했음을 의미합니다.

In [ ]:
#
# #  Isolation Forest 이상치 탐지 결과 해석
#
# # --- 3단계: 예측 및 점수 결과 (상위 5개) ---
# # (주로 정상 데이터 포인트가 포함되어 있습니다.)
#
# # Column 해석:
# # Feature1, Feature2: 모델 입력으로 사용된 두 개의 특성(좌표) 값입니다.
# # anomaly_prediction: 모델이 이 데이터 포인트를 분류한 결과입니다. (1: 정상, -1: 이상치)
# # anomaly_score: 이상치 점수입니다. 양수이고 0에 가까울수록 **정상**, 음수이고 절대값이 클수록 **강한 이상치**입니다.
#
#    Feature1  Feature2  anomaly_prediction  anomaly_score
# 0  2.149014  1.958521                   1       0.088736 # 정상 군집(X+2) 근처의 데이터. 점수가 양수이므로 정상(1)으로 분류됨.
# 1  2.194307  2.456909                   1       0.037808 # 점수가 0에 가깝지만 양수이므로 정상(1)으로 분류됨.
# 2  1.929754  1.929759                   1       0.096795 # 점수가 높으므로(0.096) 매우 확실한 정상 데이터로 판단됨.
# 3  2.473764  2.230230                   1       0.009035 # 정상 군집 내의 경계 쪽에 가까워 점수가 0에 가깝지만, 여전히 정상(1)으로 분류됨.
# 4  1.859158  2.162768                   1       0.078680 # 정상(1)으로 분류됨.
#
# # ----------------------------------------------------------------------
#
# # --- 4단계: 탐지된 이상치 확인 (점수가 낮은 순 = 가장 이상함) ---
# # (모델이 이상치(-1)로 분류한 데이터 중 가장 의심되는 순서로 정렬됨.)
#
# # Column 해석:
# # anomaly_score: 음수 값이며, 이 값의 **절댓값이 클수록** (즉, 값이 더 낮을수록) **가장 확실한 이상치**입니다.
# # anomaly_prediction: 모두 -1이며, 이는 설정된 'contamination=0.15' 임계치에 의해 이상치로 분류되었음을 의미합니다.
#
#      Feature1  Feature2  anomaly_prediction  anomaly_score
# 213 -3.755998 -3.701214                  -1      -0.300176 # 가장 낮은 점수(-0.30)를 가짐. 정상 군집에서 가장 멀리 떨어져 있어 가장 강한 이상치로 판단됨.
# 204  2.936579  3.305924                  -1      -0.252973 # 두 번째로 강한 이상치. (Feature2 값이 높음)
# 218 -3.586546  0.250837                  -1      -0.244863 # Feature1은 낮고 Feature2는 평균 근처인, 정상 패턴을 벗어난 이상치.
# 210  0.626241 -3.712462                  -1      -0.233414 # Feature1은 평균 근처, Feature2는 매우 낮은 값인 이상치.
# 208  3.120043 -1.296039                  -1      -0.232923 # 낮은 음수 점수(-0.23)를 가진 이상치.
#